In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
import pandas as pd
import statistics
from tqdm import tqdm
import numpy as np
import random
import os

/home/huihai/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352430114/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def get_time_frame(df, M, D, HH, MM):
    df_M = df[df['M'] == M]
    df_D = df_M[df_M['D'] == D]
    df_HH = df_D[df_D['HH'] == HH]
    df_MM = df_HH[df_HH['MM'] == MM]
    return df_MM

In [5]:
df_segments = pd.read_csv(r'../Dataset/New_Road_Segments_Divided_Dir.csv')
df_segments = df_segments[['SegmentID_', 'TrafDir','SegDir', 'ID']]

In [6]:
df_count_0 = pd.read_csv(r'../Dataset/Automated_Traffic_Volume_Counts.csv')

In [17]:
year = 2013

In [18]:
df_count = df_count_0[df_count_0['Yr'] == year]

In [19]:
Yr = df_count['Yr'].to_list()
M = df_count['M'].to_list()
D = df_count['D'].to_list()
time = []

for i in range(len(Yr)):
#     print(i)
    t = str(Yr[i]) + '_' + str(M[i]) + '_' + str(D[i])
    time.append(t)

time_list = list(set(time))
print('time len is: ' + str(len(time_list)))

time len is: 348


In [20]:
hour_list = list(set(df_count['HH'].to_list()))
Minute_list = list(set(df_count['MM'].to_list()))

y_2019 = []
time_record = []
for t in time_list:
    print(t)
    y = [-1]*df_segments.shape[0]
    M = int(t.split('_')[1])
    D = int(t.split('_')[2])
    HH = random.choice(hour_list)
    MM = random.choice(Minute_list)
    
    time_frame = get_time_frame(df_count, M, D, HH, MM)
    
    seg_list = time_frame['SegmentID'].to_list()
    dir_list = time_frame['Direction'].to_list()
    vol_list = time_frame['Vol'].to_list()
    
    for i in range(len(seg_list)):
        seg = seg_list[i]
        direction = dir_list[i]
        vol = vol_list[i]
        
        df_row = df_segments[df_segments['SegmentID_'] == seg]
        if df_row.shape[0] == 0:
#             print('No segment: ' + str(seg))
            continue
        else:
            if df_row['TrafDir'].values[0] != 'T':
                segment_ID = df_row['ID'].values[0]
                y[segment_ID] = vol
            else:
                df_dir = df_row[df_row['SegDir'] == direction]
                if df_dir.shape[0] == 0:
#                     print("No direction: " + str(seg))
                    continue
                else:
                    segment_ID = df_dir['ID'].values[0]
                    y[segment_ID] = vol
    a = 0
    for data in y:
        if data != -1:
            a += 1
    if a > 20:
        print(a)
        y_2019.append(y) 
        time_record.append([M, D, HH, MM])

2013_2_16
2013_3_7
38
2013_12_3
2013_6_22
95
2013_7_17
2013_6_17
106
2013_2_2
151
2013_5_6
60
2013_4_15
52
2013_9_8
2013_7_7
2013_3_21
66
2013_10_21
259
2013_8_4
2013_12_16
106
2013_6_20
127
2013_9_18
39
2013_9_16
34
2013_3_23
27
2013_11_19
121
2013_2_5
159
2013_2_17
2013_6_4
81
2013_9_6
2013_3_18
49
2013_5_5
31
2013_5_9
70
2013_3_2
33
2013_3_4
27
2013_3_8
2013_5_25
48
2013_1_17
70
2013_4_21
53
2013_12_12
112
2013_7_20
52
2013_4_1
21
2013_8_13
2013_10_24
357
2013_4_6
32
2013_6_11
93
2013_4_13
69
2013_5_11
44
2013_1_9
44
2013_9_9
2013_11_14
73
2013_8_26
2013_11_28
24
2013_1_28
48
2013_5_3
62
2013_6_19
107
2013_3_29
21
2013_9_20
39
2013_9_7
2013_3_13
55
2013_12_9
111
2013_4_12
45
2013_9_14
38
2013_9_29
54
2013_7_2
2013_4_3
21
2013_1_14
71
2013_1_13
65
2013_2_21
24
2013_11_13
61
2013_5_27
31
2013_11_23
135
2013_6_5
90
2013_12_7
93
2013_1_29
65
2013_4_7
32
2013_7_6
2013_10_11
28
2013_6_8
87
2013_9_26
58
2013_4_26
78
2013_9_19
39
2013_12_1
2013_5_17
62
2013_1_21
2013_3_12
55
2013_2_14
2013_

In [21]:
y_2019_df = pd.DataFrame(zip(*y_2019))

path = os.path.join(r'../Data/', 'y_' + str(year) + '.csv')
path_record = os.path.join(r'../Data/', 'y_record_' + str(year) + '.csv')

y_record_df = pd.DataFrame(time_record)

print(path)
print(len(y_2019))
y_2019_df.to_csv(path)
y_record_df.to_csv(path_record)

../Data/y_2013.csv
259


In [174]:
y_2019

[]